In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
df.claim.value_counts()

In [ ]:
df.shape

In [ ]:
df.describe()

# **EDA**

In [ ]:
df.isna().sum()

In [ ]:
df1 = df.copy()

In [ ]:
df1=df1.fillna(df1.mean())

In [ ]:
X = df1.iloc[:,1:-1]
y=df1.iloc[:,-1]

# **Feature Selection**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif

In [ ]:
def select_features(X, y):
	fs = SelectKBest(score_func=f_classif, k='all')
	fs.fit(X, y)
	X_fs = fs.transform(X)
	return X_fs, fs

In [ ]:
X_fs,fs = select_features(X, y)
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))

In [ ]:
features=[]
for i in range(len(fs.scores_)):
    if fs.scores_[i] >= 1:
        features.append(i)

In [ ]:
len(features)

In [ ]:
X=X.iloc[:,features]

## **Feature scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X = scalar.fit_transform(X.values)

## **Data Split**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y.values, test_size=0.2,random_state=42,shuffle=True)

# **Model Build**

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
model = XGBClassifier(n_estimators=1200,n_jobs=-1,max_depth=2,importance_type="total_gain",max_delta_step=2)
model.fit(X_train ,y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))
print(accuracy_score(y_test,preds))
print(metrics.roc_auc_score(y_test,preds))

# **Test**

In [ ]:
df_test.info()

In [ ]:
df_test.isna().sum()

In [ ]:
df_test1 = df_test.copy()
df_test1 = df_test1.fillna(df_test1.mean())

In [ ]:
t = df_test1.iloc[:,1:]

In [ ]:
test_X = t.iloc[:,features]

In [ ]:
test_X=scalar.fit_transform(test_X)

In [ ]:
y_pred = model.predict_proba(test_X)[:,1]

In [ ]:
y_pred

In [ ]:
subm = pd.DataFrame(df_test1['id'])
subm['claim'] = y_pred

In [ ]:
subm

In [ ]:
subm.to_csv('submit.csv',index=False)